In [2]:
from bs4 import BeautifulSoup
import requests
!wget https://raw.githubusercontent.com/MBUYt0n/tf2torch/main/links.txt

--2025-02-04 16:32:33--  https://raw.githubusercontent.com/MBUYt0n/tf2torch/main/links.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305785 (299K) [text/plain]
Saving to: ‘links.txt’

links.txt           100%[===================>] 298.62K  --.-KB/s    in 0.004s  

2025-02-04 16:32:34 (75.7 MB/s) - ‘links.txt’ saved [305785/305785]



In [3]:
with open("/content/links.txt", "r") as f:
    links = f.readlines()
    l, k = [], []
    for i in links:
        try:
            l.append("https://www.tensorflow.org/" + i.strip().split("file:///")[1])
        except:
            l.append(i.split(". ")[1])

In [4]:
l = l[12:]

In [4]:
l[3]

'https://www.tensorflow.org/api_docs/python/tf/GradientTape'

In [12]:
!pip install kdbai-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 8.0 MB/s eta 0:00:00


In [13]:
import kdbai_client as kdbai
session = kdbai.Session(endpoint="https://cloud.kdb.ai/instance/kgeujvgj3s", api_key="2a654c56f4-v0OPGUcyQX+Ej9AfVXsAOXYn+jKW6KeePnscNNFYcN6THu0YXfvKeG1Qfockuy8441Psb58z1wKI06mV")

db = session.database('default')

In [32]:
print(db.tables)

[]


In [33]:
schema = [{'name': 'id', 'type': 'int16'},
{'name': 'header', 'type': 'str'},
{'name': 'vectors', 'type': 'float32s'}]

indexes = [ {'name': 'vectorIndex', 'type': 'flat',
'params': {'dims': 768, 'metric': 'CS'},
'column': 'vectors'}]

In [34]:
table = db.create_table('test',schema=schema, indexes=indexes)

In [19]:
data = {}
try:
        r = requests.get(l[0], allow_redirects=False)
        soup = BeautifulSoup(r.content)
        a = soup.find_all("article", class_="devsite-article")[0].text.split()
        data[a[5]] = ' '.join(a[6:])
except Exception as e:
        print(e)

In [20]:
len(data["tf.AggregationMethod"].split(" "))

220

In [21]:
from transformers import BertModel, BertTokenizer
import torch

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [22]:
inp = tokenizer(data["tf.AggregationMethod"], return_tensors="pt", padding=True, truncation=True)

In [23]:
with torch.no_grad():
    outputs = model(**inp)
last_hidden_state = outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)
last_hidden_state.shape

torch.Size([1, 401, 768])

In [35]:
import pandas as pd

d = pd.DataFrame({
    "id" : [i for i in range(401)],
    "header" : ["tf.AggregationMethod" for i in range(401)],
    "vectors" : [i for i in last_hidden_state.numpy()[0]]
})

In [36]:
db.table("test").insert(d)

{'rowsInserted': 401}

In [37]:
inp = tokenizer("EXPERIMENTAL_TREE", return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inp).last_hidden_state[0]

In [42]:
outputs.shape

torch.Size([5, 768])

In [ ]:
results = db.table("test").search({'vectorIndex': outputs.numpy()}, n=1)
results[0]